## Airtable tests

In [8]:
import requests

# Apenas para deixar os dados escondidos
import os
from dotenv import find_dotenv, load_dotenv
load_dotenv(find_dotenv())


airtable_api = os.environ.get('AIRTABLE_KEY')
airtable_fonte_api_url = 'https://api.airtable.com/v0/appCmmcOxNBLaO89F/Table%201'
facebook_token = os.environ.get('FACEBOOK_TOKEN')

In [9]:
# Lendo a tabela
url = f'{airtable_fonte_api_url}?api_key={airtable_api}'
requisicao = requests.get(url)
veiculos = requisicao.json()

In [12]:
veiculos['records']

[{'createdTime': '2018-02-19T18:15:59.713Z',
  'fields': {'Name': 'O globo', 'facebook_id': 115230991849922},
  'id': 'rec4u7Iv5r1BsFpO9'},
 {'createdTime': '2018-02-19T18:24:00.159Z',
  'fields': {'Name': 'Estadão', 'facebook_id': 12312312321},
  'id': 'rec51V8Qlxb2abmx9'},
 {'createdTime': '2018-02-19T18:15:59.713Z',
  'fields': {'Name': 'Folha de S. Paulo', 'facebook_id': 100114543363891},
  'id': 'recDT6XepDD6b7quL'}]

In [ ]:
# Transformando em dicionários

In [13]:
def dados_airtable(url, api_key):
    url = f'{airtable_fonte_api_url}?api_key={airtable_api}'
    requisicao = requests.get(url)
    veiculos = requisicao.json()
    dados = []
    for v in veiculos['records']:
        dados.append(v['fields'])
    return dados
        

In [15]:
dados_airtable(airtable_fonte_api_url, airtable_api)

[{'Name': 'O globo', 'facebook_id': 115230991849922},
 {'Name': 'Estadão', 'facebook_id': 12312312321},
 {'Name': 'Folha de S. Paulo', 'facebook_id': 100114543363891}]

In [ ]:
# Passando a limpo

-----

### Ligando o AirTable a uma função

In [16]:
# o que fazer com uma página de Facebook? Criar um teste
# Pegar o feed
facebook_id = 100114543363891
url = f'https://graph.facebook.com/v2.11/{facebook_id}/feed?access_token={facebook_token}'
feed = requests.get(url).json()['data']

In [18]:
# Pegar os dados individuais 
facebook_id = '100114543363891_2275504752491515'
fields = 'attachments,comments.summary(true),created_time,description,link,message,permalink_url,shares,type'
url = f'https://graph.facebook.com/v2.11/{facebook_id}?fields={fields}&access_token={facebook_token}'
r = requests.get(url).json()
r

{'attachments': {'data': [{'description': ' ',
    'media': {'image': {'height': 600,
      'src': 'https://external.xx.fbcdn.net/safe_image.php?d=AQD4VJuDNqdCcQUG&w=600&h=600&url=http%3A%2F%2Fd133mo2u8ekg8z.cloudfront.net%2Fwp-content%2Fuploads%2F2018%2F02%2F07fev18fhchuck.jpg&cfs=1&sx=497&sy=0&sw=600&sh=600&_nc_hash=AQDiwFt-bWM0-Z73',
      'width': 600}},
    'target': {'url': 'https://l.facebook.com/l.php?u=http%3A%2F%2Fpiaui.folha.uol.com.br%2Fc%2Fhuck-vai-conversar-com-fhc-sobre-candidatura%2F%3Futm_source%3Dfacebook%26utm_medium%3Dsocial%26utm_campaign%3Dfbfolha&h=ATOgiaJsz_S4vmv341n7siFOBNWp7hUPVFUGOnPECoDVZOz0rMzTb-RgnHTHLCP9JDoONV18OeZx00g2yHhLvjwhBV1R5KEyOV-kNBdUpl7rdl0eMiFe9d3ZOCREYiOxDx6TvwCIILYsKUtV&s=1'},
    'title': 'Huck vai conversar com FHC sobre candidatura nesta quinta',
    'type': 'share',
    'url': 'https://l.facebook.com/l.php?u=http%3A%2F%2Fpiaui.folha.uol.com.br%2Fc%2Fhuck-vai-conversar-com-fhc-sobre-candidatura%2F%3Futm_source%3Dfacebook%26utm_medium%3Dsoc

In [40]:
# Transformar em função

def get_feed(page_id, facebook_token):
    url = f'https://graph.facebook.com/v2.11/{page_id}/feed?access_token={facebook_token}'
    feed = requests.get(url).json()['data']
    lista_de_posts = []
    for entrada in feed:
        lista_de_posts.append(entrada['id'])
    return lista_de_posts

def get_post_stats(post_id, facebook_token):
    # Pegar os dados individuais 
    fields = 'attachments,comments.summary(true),created_time,description,link,message,permalink_url,shares,type'
    url = f'https://graph.facebook.com/v2.11/{post_id}?fields={fields}&access_token={facebook_token}'
    r = requests.get(url).json()
    return r

def get_objects(page):
    posts = get_feed(page, facebook_token)
    dados = []
    for post in posts:
        info = get_post_stats(post, facebook_token)
        dados.append(info)
    for d in dados:
        print(tem_intervencao(d))
        
# for loop com diferentes páginas (print)
def tem_intervencao(post):
    if 'message' in post:
        if 'intervenção' in post['message']:
            return {"mensagem": post['message'], 
                    "link": post['permalink_url']}



----

### Inserindo os dados na AirTable

In [45]:
# Criando novas linhas

airtable_destino_api_url = 'https://api.airtable.com/v0/appS6w7do1Xy0skha/Table%201'
headers = {"Authorization": f'Bearer {airtable_api}'}
new_content = {"fields": {"mensagem": "isso é um teste", "link": "http://www.google.com"}} # Dicionário com as informações da nova linha
s = requests.post(airtable_destino_api_url, json=new_content, headers=headers)

In [ ]:
def adicionar_linha(novos_dados):
    airtable_destino_api_url = 'https://api.airtable.com/v0/appS6w7do1Xy0skha/Table%201'
    headers = {"Authorization": f'Bearer {airtable_api}'}
    new_content = {"fields": novos_dados}
    s = requests.post(airtable_destino_api_url, json=new_content, headers=headers)
    print('dados_acrescentados')

In [43]:
s.content

b'{"error":{"type":"INVALID_REQUEST_MISSING_FIELDS","message":"Could not find field \\"fields\\" in the request body"}}'

In [ ]:
# Rodando a função anterior, armazenando o código

In [ ]:
# Se não houver uma entrada anterior, acrescentar

In [48]:
# Organizando tudo em um arquivo .py

----

**Snippets úteis:**

In [9]:
# Pegar o feed
facebook_id = ''
url = f'https://graph.facebook.com/v2.11/{facebook_id}/feed?access_token={facebook_token}'
feed = requests.get(url).json()['data']

In [ ]:
# Pegar os dados individuais 
facebook_id = ''
fields = 'attachments,comments.summary(true),created_time,description,link,message,permalink_url,shares,type'
url = f'https://graph.facebook.com/v2.11/{facebook_id}?fields={fields}&access_token={facebook_token}'
r = requests.get(url).json()